In [25]:
import jax
import jax.numpy as jnp
import time

# --- 1. JAX Setup ---
key = jax.random.PRNGKey(42)

# --- 2. Imports from the 'chewc' library ---
from chewc.sp import SimParam
from chewc.population import Population, msprime_pop # Use msprime_pop
from chewc.trait import add_trait_a
from chewc.burnin import run_burnin
# Import the generation runner
from chewc.pipe import run_generation


There are two functions to simulate founder genomes

1) quickHaplo

    this simulate a randomized population where each haplotype generated is 1/0. This creates founders with high amounts of genetic diversity and no population structure.

2) msprime_pop

    this function uses the msprime python library to create populations with population structures generated through a separate coalescent simulation

In [49]:
from dataclasses import field
from typing import List, Optional, Dict, Callable

from flax.struct import dataclass as flax_dataclass # Using flax's dataclass for JAX-friendliness
import jax
import jax.numpy as jnp

# from chewc.sp import SimParam

import msprime
import tskit
import numpy as np
import random
from collections import defaultdict

key, pop_key = jax.random.split(key)
nInd = 5
nChr = 2
segSites = 4

founder_genomes = msprime_pop(pop_key, nInd, segSites, nChr)
founder_genomes

Population(nInd=5, nTraits=0, has_ebv=No)

In [50]:
# The founder_genomes object already contains the genetic map and genotype info
SP = SimParam(founderPop=founder_genomes)

In [51]:
SP

SimParam(nChr=2, nTraits=0, ploidy=2, sexes='no')

In [52]:
# Access the founder pop in the Sim Params
SP.founderPop

Population(nInd=5, nTraits=0, has_ebv=No)

In [53]:
# Get the number of individuals in the founder pop
SP.founderPop.nInd

5

In [54]:
# Get the individual ID's 
SP.founderPop.id

Array([0, 1, 2, 3, 4], dtype=int32)

In [55]:
SP.founderPop.nChr

2

In [56]:
SP.founderPop.geno.shape

(5, 2, 2, 4)

In [58]:
SP.founderPop.geno

Array([[[[1, 1, 0, 0],
         [1, 1, 1, 0]],

        [[0, 0, 0, 0],
         [0, 1, 0, 0]]],


       [[[1, 0, 0, 1],
         [0, 1, 1, 0]],

        [[0, 0, 0, 0],
         [0, 0, 0, 0]]],


       [[[1, 0, 1, 0],
         [0, 0, 1, 0]],

        [[0, 1, 1, 0],
         [1, 0, 0, 1]]],


       [[[0, 1, 1, 1],
         [0, 0, 1, 0]],

        [[0, 1, 1, 0],
         [0, 0, 0, 0]]],


       [[[1, 0, 1, 0],
         [1, 0, 0, 0]],

        [[0, 0, 1, 0],
         [1, 1, 1, 0]]]], dtype=uint8)

In [57]:
SP.founderPop.dosage

Array([[2, 2, 1, 0, 0, 1, 0, 0],
       [1, 1, 1, 1, 0, 0, 0, 0],
       [1, 0, 2, 0, 1, 1, 1, 1],
       [0, 1, 2, 1, 0, 1, 1, 0],
       [2, 0, 1, 0, 1, 1, 2, 0]], dtype=uint32)